In [ ]:
import pandas as pd
data_path = '../data/raw/synthetic_disease_symptoms.csv'
df = pd.read_csv(data_path)

if not df.empty:
    X = df.drop('diseases', axis=1)
    y_text = df['diseases']
    print("Đã tách X (đặc trưng) và y_text (nhãn dạng text).")
    print("Kích thước của X:", X.shape)
    print("Kích thước của y_text:", y_text.shape)
else:
    print("DataFrame rỗng, không thể tiếp tục.")
    # Dừng hoặc xử lý lỗi

Đã tách X (đặc trưng) và y_text (nhãn dạng text).
Kích thước của X: (246945, 377)
Kích thước của y_text: (246945,)


In [2]:
from sklearn.preprocessing import LabelEncoder
import joblib # Để lưu encoder

if 'y_text' in locals() and not y_text.empty: # Kiểm tra y_text tồn tại và không rỗng
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_text) # Fit và transform cột tên bệnh

    print("\nĐã mã hóa nhãn 'Disease' thành dạng số.")
    print("Ví dụ 5 giá trị y sau khi mã hóa:", y[:5])
    print("Các lớp bệnh duy nhất mà encoder đã học:", label_encoder.classes_[:10]) # In ra 10 lớp đầu tiên

    # Lưu LabelEncoder
    model_dir = '../models/' # Đảm bảo thư mục này tồn tại
    import os
    os.makedirs(model_dir, exist_ok=True) # Tạo thư mục nếu chưa có
    joblib.dump(label_encoder, os.path.join(model_dir, 'disease_label_encoder.pkl'))
    print(f"LabelEncoder đã được lưu vào {os.path.join(model_dir, 'disease_label_encoder.pkl')}")
else:
    print("Không có y_text để mã hóa.")


Đã mã hóa nhãn 'Disease' thành dạng số.
Ví dụ 5 giá trị y sau khi mã hóa: [531 531 531 531 531]
Các lớp bệnh duy nhất mà encoder đã học: ['abdominal aortic aneurysm' 'abdominal hernia' 'abscess of nose'
 'abscess of the lung' 'abscess of the pharynx' 'acanthosis nigricans'
 'acariasis' 'achalasia' 'acne' 'actinic keratosis']
LabelEncoder đã được lưu vào ../models/disease_label_encoder.pkl


In [4]:
# Trước khi gọi train_test_split
if 'y_text' in locals() and not y_text.empty:
    value_counts = y_text.value_counts()
    to_remove = value_counts[value_counts < 2].index # Lấy tên các bệnh có ít hơn 2 mẫu

    if len(to_remove) > 0:
        print(f"\nCác bệnh sau sẽ bị loại bỏ do có ít hơn 2 mẫu: {list(to_remove)}")
        # Lọc bỏ các hàng có bệnh nằm trong danh sách to_remove
        df_filtered = df[~df['diseases'].isin(to_remove)].copy() # Tạo bản sao để tránh SettingWithCopyWarning

        if not df_filtered.empty:
            X = df_filtered.drop('diseases', axis=1)
            y_text_filtered = df_filtered['diseases']

            # Mã hóa lại y_text_filtered
            label_encoder = LabelEncoder() # Tạo lại hoặc dùng lại encoder nếu bạn muốn giữ mapping cũ (phức tạp hơn)
            y = label_encoder.fit_transform(y_text_filtered)

            # Lưu lại label_encoder MỚI
            joblib.dump(label_encoder, os.path.join(model_dir, 'disease_label_encoder_filtered.pkl'))
            print(f"LabelEncoder MỚI (đã lọc) đã được lưu vào {os.path.join(model_dir, 'disease_label_encoder_filtered.pkl')}")


Các bệnh sau sẽ bị loại bỏ do có ít hơn 2 mẫu: ['turner syndrome', 'rocky mountain spotted fever', 'open wound of the cheek', 'high blood pressure', 'open wound due to trauma', 'open wound of the chest', 'huntington disease', 'open wound of the knee', 'foreign body in the nose', 'diabetes', 'thalassemia', 'heat stroke', 'gas gangrene', 'typhoid fever', 'open wound of the head', 'myocarditis', 'chronic ulcer', 'hypergammaglobulinemia', 'kaposi sarcoma']
LabelEncoder MỚI (đã lọc) đã được lưu vào ../models/disease_label_encoder_filtered.pkl


In [6]:
# Trước khi gọi train_test_split
if 'y_text' in locals() and not y_text.empty:
    value_counts = y_text.value_counts()
    to_remove = value_counts[value_counts < 2].index # Lấy tên các bệnh có ít hơn 2 mẫu

    if len(to_remove) > 0:
        print(f"\nCác bệnh sau sẽ bị loại bỏ do có ít hơn 2 mẫu: {list(to_remove)}")
        # Lọc bỏ các hàng có bệnh nằm trong danh sách to_remove
        df_filtered = df[~df['diseases'].isin(to_remove)].copy() # Tạo bản sao để tránh SettingWithCopyWarning

        if not df_filtered.empty:
            X = df_filtered.drop('diseases', axis=1)
            y_text_filtered = df_filtered['diseases']

            # Mã hóa lại y_text_filtered
            label_encoder = LabelEncoder() # Tạo lại hoặc dùng lại encoder nếu bạn muốn giữ mapping cũ (phức tạp hơn)
            y = label_encoder.fit_transform(y_text_filtered)

            # Lưu lại label_encoder MỚI
            joblib.dump(label_encoder, os.path.join(model_dir, 'disease_label_encoder_filtered.pkl'))
            print(f"LabelEncoder MỚI (đã lọc) đã được lưu vào {os.path.join(model_dir, 'disease_label_encoder_filtered.pkl')}")
            
            from sklearn.model_selection import train_test_split

# Bây giờ mới thực hiện train_test_split với X và y đã lọc
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                test_size=0.2,
                random_state=42,
                stratify=y
            )
            print("\nĐã chia dữ liệu thành tập huấn luyện và tập kiểm thử:")
            print("Kích thước X_train:", X_train.shape)
            print("Kích thước X_test:", X_test.shape)
            print("Kích thước y_train:", y_train.shape)
            print("Kích thước y_test:", y_test.shape)

                        # Kiểm tra phân phối lớp trong tập train và test (tùy chọn nhưng hữu ích)
            print("\nPhân phối lớp trong y_train:")
            print(pd.Series(y_train).value_counts(normalize=True).head())
            print("\nPhân phối lớp trong y_test:")
            print(pd.Series(y_test).value_counts(normalize=True).head())
               
    else:
            print("Sau khi lọc, DataFrame rỗng. Kiểm tra lại ngưỡng lọc.")
else:
        print("\nKhông có bệnh nào cần loại bỏ (tất cả các bệnh đều có ít nhất 2 mẫu).")
        # Nếu không có gì để loại bỏ, bạn có thể tiến hành train_test_split với X và y ban đầu (sau khi mã hóa y_text)
        # (Đảm bảo y đã được mã hóa từ y_text không bị lọc)
        # y = label_encoder.fit_transform(y_text) # Mã hóa lại y_text gốc nếu chưa làm hoặc nếu có thay đổi
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, # y này là y đã mã hóa từ y_text gốc
            test_size=0.2,
            random_state=42,
            stratify=y
        )
        print("\nĐã chia dữ liệu (không lọc) thành tập huấn luyện và tập kiểm thử.")
        print("Kích thước X_train:", X_train.shape)
        print("Kích thước X_test:", X_test.shape)
        print("Kích thước y_train:", y_train.shape)
        print("Kích thước y_test:", y_test.shape)


Các bệnh sau sẽ bị loại bỏ do có ít hơn 2 mẫu: ['turner syndrome', 'rocky mountain spotted fever', 'open wound of the cheek', 'high blood pressure', 'open wound due to trauma', 'open wound of the chest', 'huntington disease', 'open wound of the knee', 'foreign body in the nose', 'diabetes', 'thalassemia', 'heat stroke', 'gas gangrene', 'typhoid fever', 'open wound of the head', 'myocarditis', 'chronic ulcer', 'hypergammaglobulinemia', 'kaposi sarcoma']
LabelEncoder MỚI (đã lọc) đã được lưu vào ../models/disease_label_encoder_filtered.pkl

Đã chia dữ liệu thành tập huấn luyện và tập kiểm thử:
Kích thước X_train: (197540, 377)
Kích thước X_test: (49386, 377)
Kích thước y_train: (197540,)
Kích thước y_test: (49386,)

Phân phối lớp trong y_train:
164    0.004936
471    0.004931
137    0.004931
747    0.004931
653    0.004926
Name: proportion, dtype: float64

Phân phối lớp trong y_test:
164    0.004941
747    0.004941
471    0.004941
343    0.004920
653    0.004920
Name: proportion, dtype:

In [7]:
# (Ở cuối notebook tiền xử lý)
# Đảm bảo thư mục processed tồn tại
import os
os.makedirs('../data/processed/', exist_ok=True)

pd.DataFrame(X_train, columns=X.columns).to_csv('../data/processed/X_train_filtered.csv', index=False)
pd.DataFrame(X_test, columns=X.columns).to_csv('../data/processed/X_test_filtered.csv', index=False)
pd.Series(y_train).to_csv('../data/processed/y_train_filtered.csv', index=False, header=['Disease_encoded'])
pd.Series(y_test).to_csv('../data/processed/y_test_filtered.csv', index=False, header=['Disease_encoded'])
print("Đã lưu các tập dữ liệu X_train, X_test, y_train, y_test đã lọc.")

Đã lưu các tập dữ liệu X_train, X_test, y_train, y_test đã lọc.
